In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV #La Grilla que realiza El cross y prueba los hiperparametros
from sklearn.svm import SVC #Suport vector machine modelo de clasificacion
from sklearn.linear_model import ElasticNetCV # modelo de clasificacion
from sklearn.linear_model import SGDClassifier # modelo de clasificacion
from sklearn.neighbors import KNeighborsClassifier # modelo de clasificacion
from sklearn.tree import DecisionTreeClassifier # modelo de clasificacion

In [2]:
df = pd.read_csv("Datos/fifa_eda_stats.csv")
df = df[df['Club'].isin(df.Club.unique()[[0,20,100,250]])]
x = df.select_dtypes(['number'])
y = df['Club']

In [3]:
x = x.dropna()
x = x[['Overall','Potential','International Reputation', 'ShortPassing','LongPassing','Reactions','Vision','Composure']]
y = y.dropna()

In [4]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=20,random_state=123)

# SVC

In [5]:
Svc_Model = SVC(random_state=123)
parametros_Svc = {'C':np.arange(0.5,5,0.5), 'degree':[3,4,5,6,7]}
grid_svc = GridSearchCV(estimator = Svc_Model,param_grid = parametros_Svc,scoring = 'accuracy', cv = 5, verbose = 1)
grid_svc.fit(X_train, y_train)
svc = pd.DataFrame(grid_svc.cv_results_)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


In [6]:
svc['mean_test_score'].sort_values(ascending=False).head(5)

33    0.481429
36    0.481429
30    0.481429
31    0.481429
32    0.481429
Name: mean_test_score, dtype: float64

# SDGClassifier

In [7]:
#SDG_Model = SGDClassifier()
#parametros_SDG = {'C':range(0.5,5,0.5), 'degree':[3,4,5,6,7]}
#grid_SDG = GridSearchCV(estimator = SDG_Model,param_grid = parametros_SDG,scoring = 'accuracy', cv = 5, verbose = 1)

# Regresion lineal

# KNN

In [8]:
KNN_Model = KNeighborsClassifier()
parametros_KNN = {'n_neighbors':np.arange(3,8,1), 'weights':['uniform','distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
grid_KNN = GridSearchCV(estimator = KNN_Model,param_grid = parametros_KNN,scoring = 'accuracy', cv = 5, verbose = 1)
grid_KNN.fit(X_train, y_train)
KNN = pd.DataFrame(grid_KNN.cv_results_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


In [9]:
KNN['mean_test_score'].sort_values(ascending=False).head(5)

25    0.559048
35    0.559048
5     0.559048
15    0.559048
34    0.558095
Name: mean_test_score, dtype: float64

In [13]:
grid_KNN.best_params_

{'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}

In [10]:
modelo_KNN_best = grid_KNN.best_estimator_
modelo_KNN_best

KNeighborsClassifier(weights='distance')

In [20]:
KNN_desvio = grid_KNN.cv_results_['std_test_score']

In [22]:
index = grid_KNN.best_index_
KNN_mejor_desvio = KNN_desvio[index]
KNN_mejor_desvio 

In [24]:
print("Mejor Accuracy: ",grid_KNN.best_score_,' +/- ', 2*KNN_mejor_desvio, '( 95% )')

Mejor Accuracy:  0.559047619047619  +/-  0.15029524986345796 ( 95% )


# Arbol de decision 

In [11]:
Arbol_Model = DecisionTreeClassifier(random_state=0)
parametros_arbol = {'criterion':['gini', 'entropy', 'log_loss'],'splitter': ['best', 'random']}
grid_arbol = GridSearchCV(estimator = Arbol_Model,param_grid = parametros_arbol,scoring = 'accuracy', cv = 5, verbose = 1)
grid_arbol.fit(X_train, y_train)
arbol = pd.DataFrame(grid_arbol.cv_results_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\Alumno\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Alumno\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Alumno\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 903, in fit
    super().fit(
  File "c:\Users\Alumno\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 348, in fit
    criterion = CRITERIA_CLF[self.criterion](self.n_outputs_,
KeyError: 'log_loss'

  warnings.warn("Estimator fit failed. The score on this train-test"
c:\Users\Alumno\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Detail

In [12]:
arbol['mean_test_score'].sort_values(ascending=False).head(5)

1    0.520476
3    0.501429
2    0.490000
0    0.480952
4         NaN
Name: mean_test_score, dtype: float64

# Entrenamiento Con todos los datos de X e Y Al modelo de KNN 

In [31]:
KNN_Model_1 = KNeighborsClassifier(weights='distance')
KNN_Model_1.fit(X_train,y_train)
KNN_Model_1.predict(X_train)
KNN_Model_1.score(X_test,y_test)

0.45

In [32]:
KNN_Modelo_final = KNN_Model_1 = KNeighborsClassifier(weights='distance')
KNN_Modelo_final.fit(x,y)

KNeighborsClassifier(weights='distance')

In [33]:
import pickle
# Comenté las siguientes líneas porque sino cada vez que corro este archivo, guardaría.

archivo = 'Datos/modelo_KNN.sav'
pickle.dump(KNN_Modelo_final, open(archivo, 'wb'))

archivo = 'Datos/modelo_KNN.sav'
modelo_cargado=pickle.load(open(archivo, 'rb'))